In [1]:
import pandas as pd
import dash
from dash import dcc
from dash import html
import plotly.graph_objs as go

In [2]:
df = pd.read_csv('dataset_space_API_past_launches_raw.csv')

df = df.drop(['FlightNumber', 'Date', 'Longitude', 'Latitude'], axis=1) # 删掉没用的

# Encode Outcome
df = df.replace({'Outcome': {'True Ocean': 1, 
                             'True RTLS': 1,
                             'True ASDS': 1,
                             'False Ocean':0,
                             'False RTLS':0,
                             'False ASDS':0,
                             'None ASDS':0,
                             'None None':0,},
                 'GridFins': {True: 1, False: 0},
                 'Reused': {True: 1, False: 0},
                 'Legs': {True: 1, False: 0}
                })

# Missing Values
PayloadMass_mean = df['PayloadMass'].mean()
PayloadMass_mean = round(PayloadMass_mean, 1)
df['PayloadMass'] = df['PayloadMass'].fillna(PayloadMass_mean)
df['LandingPad'] = df['LandingPad'].fillna('missing')
df['Block'] = df['Block'].fillna('0')

In [ ]:
app = dash.Dash(__name__)

app.layout = html.Div(children=[
    html.H1(children='PayloadMass vs. Outcome'),

    # set the slider to control the PayloadMass
    dcc.RangeSlider(
        id = 'range-slider',
        min = df['PayloadMass'].min(),
        max = df['PayloadMass'].max(),
        step = 500,
        marks = {i: str(i) for i in range(0, int(df['PayloadMass'].max()), 500)},
        value = [df['PayloadMass'].min(), df['PayloadMass'].max()]
    ),
    dcc.Graph(id = 'graph')
])

@app.callback(
    dash.dependencies.Output('graph', 'figure'),
    [dash.dependencies.Input('range-slider', 'value')]
)

def update_graph(payload_range):
    filtered_df = df[(df['PayloadMass'] >= payload_range[0]) & (df['PayloadMass'] <= payload_range[1])]

    # create the scatter chart
    scatter_plot = go.Figure(
        data = [go.Scatter(
            x = filtered_df['PayloadMass'],
            y = filtered_df['Outcome'],
            mode = 'markers'
        )]
    )
    scatter_plot.update_layout(
        title='Outcome of diff PayloadMass',
        xaxis_title = 'PayloadMass',
        yaxis_title = 'Outcome (1 is success, 0 is failure)',
        height = 200
    )
    return scatter_plot

if __name__ == '__main__':
    app.run_server(port=8051, debug=False)

Dash is running on http://127.0.0.1:8051/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8051/ (Press CTRL+C to quit)
